<a href="https://colab.research.google.com/github/BDH-teacher/Deep_Learning_Audit_code/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Vanilla RNN Gradient Flow - Gradient Clipping

import torch
import torch.nn as nn

torch.manual_seed(0)

grad = torch.randn(100)
threshold = 5.0

grad_norm = torch.norm(grad, p=2)
print("Before clipping grad_norm:", grad_norm.item())

if grad_norm > threshold:
    grad *= (threshold / grad_norm)

grad_norm_after = torch.norm(grad, p=2)
print("After  clipping grad_norm:", grad_norm_after.item())

Before clipping grad_norm: 10.251128196716309
After  clipping grad_norm: 4.999999523162842


In [ ]:
# PyTorch API - RNN

import torch
import torch.nn as nn

# Define RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()

        # Define the RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers,
                          batch_first=True)

        # Define the fully connected layer to produce outputs
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initial hidden state of zeros
        h0 = torch.zeros(self.rnn.num_layers, x.size(0),
                         self.rnn.hidden_size).to(x.device)

        # Forward propagate the RNN
        out, hidden = self.rnn(x, h0)

        # Take the output from the last time step
        out = self.fc(out[:, -1, :])
        return out


# Example usage
input_size = 10  # Number of input features per time step
hidden_size = 20  # Number of features in the hidden state
num_layers = 2  # Number of RNN layers (stacked)
output_size = 1  # Number of output classes

# Create RNN model
model = RNNModel(input_size, \
                 hidden_size, num_layers, output_size)

# Example input (batch_size, seq_length, input_size)
x = torch.randn(5, 15, input_size)

# Forward pass through the model
output = model(x)
print("RNNModel output.shape:", output.shape)  # [batch_size, output_size]

RNNModel output.shape: torch.Size([5, 1])


In [ ]:
# PyTorch API - Bidirectional RNN

import torch
import torch.nn as nn

# Define RNN model
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(BiRNN, self).__init__()

        # Define the bidirectional RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers,
                          bidirectional=True, batch_first=True)

        # Define the output layer (since bidirectional RNN doubles the hidden size)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):  # x: (batch_size, seq_length, input_size)
        # Get the outputs from the bidirectional RNN
        # output: (batch_size, seq_length, hidden_size * 2), hidden: (num_layers * 2, batch_size, hidden_size)
        output, hidden = self.rnn(x)

        # Take the output at the last time step (for classification tasks)
        # The output from both directions is concatenated, so we access the last step from the forward and backward RNNs
        out = output[:, -1, :]  # (batch_size, hidden_size * 2)

        # Pass through the fully connected layer to get the final output
        out = self.fc(out)  # (batch_size, output_size)
        return out


bi_model = BiRNN(input_size=10, hidden_size=20, output_size=1, num_layers=2)
x_bi = torch.randn(5, 15, 10)
out_bi = bi_model(x_bi)
print("BiRNN output.shape:", out_bi.shape)  # (batch_size, output_size)

BiRNN output.shape: torch.Size([5, 1])
